In [1]:
!pip install kafka-python

/usr/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point
     |████████████████████████████████| 246 kB 1.3 MB/s eta 0:00:01


In [2]:
import json
import time 

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [7]:
t0 = time.time()

topic_name = 'test-topic'

for i in range(10):
    message = {'number': i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

Sent: {'number': 0}
Sent: {'number': 1}
Sent: {'number': 2}
Sent: {'number': 3}
Sent: {'number': 4}
Sent: {'number': 5}
Sent: {'number': 6}
Sent: {'number': 7}
Sent: {'number': 8}
Sent: {'number': 9}
took 0.51 seconds
took 0.51 seconds


In [8]:
import pandas as pd

In [10]:
!wget -nc https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz

File ‘green_tripdata_2019-10.csv.gz’ already there; not retrieving.



In [13]:
df_green = pd.read_csv("green_tripdata_2019-10.csv.gz", usecols=[
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
])

In [14]:
df_green.head

<bound method NDFrame.head of        lpep_pickup_datetime lpep_dropoff_datetime  PULocationID  DOLocationID  \
0       2019-10-01 00:26:02   2019-10-01 00:39:58           112           196   
1       2019-10-01 00:18:11   2019-10-01 00:22:38            43           263   
2       2019-10-01 00:09:31   2019-10-01 00:24:47           255           228   
3       2019-10-01 00:37:40   2019-10-01 00:41:49           181           181   
4       2019-10-01 00:08:13   2019-10-01 00:17:56            97           188   
...                     ...                   ...           ...           ...   
476381  2019-10-31 23:30:00   2019-11-01 00:00:00            65           102   
476382  2019-10-31 23:03:00   2019-10-31 23:24:00           129           136   
476383  2019-10-31 23:02:00   2019-10-31 23:23:00            61           222   
476384  2019-10-31 23:42:00   2019-10-31 23:56:00            76            39   
476385  2019-10-31 23:23:00   2019-10-31 23:56:00            56           215  

In [16]:
t0 = time.time()

topic_name = 'green-trips'

for row in df_green.itertuples(index=False):
    row_dict = {col: getattr(row, col) for col in row._fields}
    # print(row_dict)

    producer.send(topic_name, value=row_dict)

producer.flush()

t1 = time.time()
print(f'took {(t1 - t0):.2f} seconds')

took 75.07 seconds
